In [6]:
import numpy as np

def bezier(points, precision):
    """Função que cria as curvas Bezier         

    Parâmetros:
    -----------
    points: ndarray
        vetor que contém os pontos de controle
    precision: int
        número de pontos que serão criados entre o ponto inicial e final
    Retorno:
    -----------
    B: ndarray
                
    """    
    # gera a quantidade de números no intervalo de 0 e 1, dependendo do valor de precision.
    ts = np.linspace(0, 1, precision)    
    # gera uma matriz com duas colunas e a quantidade de linhas dependendo do tamanho de ts de zeros, com o tipo float
    B = np.zeros((len(ts), 2), dtype=np.float64)   
    n = len(points) - 1
    
    for idx, t in enumerate(ts):        
        for i in range(n+1):           
            bin_coef = np.math.factorial(n) / (np.math.factorial(i) * np.math.factorial(n-i))
            Pin = bin_coef * (1-t)**(n-i) * t**i
            B[idx] += Pin * points[i]
    # retorna em B todos os pontos mutliplicado por Pin 
    return B

# elemento utilizado para não ultrapassar a borda
padding = 20

def criando_pontos(tam):
    points = []    
    while len(points)<tam:
        # sorteio dos pontos
        p1x = np.random.randint(0,1776-padding)   
        p1y = np.random.randint(0,1504-padding)   
        p2x = np.random.randint(0,1776-padding) 
        p2y = np.random.randint(0,1504-padding) 

        # distância Euclidiana entre as coordenadas inicial e final
        distancia = np.sqrt((p1x-p2x)**2 + (p1y-p2y)**2)

        # só acrescenta no array de pontos elementos cuja distância seja maior que 500 pixels e menor que 1300
        if distancia > 500 and distancia < 1300:
            p1 = np.array((p1x,p1y))
            p2 = np.array((p2x,p2y))

            # empilha os pontos
            p = np.vstack((p1, p2))        
            points.append(p)
            
    # retorna os pontos criados ao longo do traçado e a distância. O elemento distância servirá de base para estipular
    # o comprimento (quantidade de colunas) dos mapas transformados.
    return points, distancia

def criar_array_curves(pontos):
    array_curves = []
    ps = pontos[0][0] # ponto inicial   
    pe = pontos[0][1] # ponto final 
    dx = pe[0]-ps[0]
    dy = pe[1]-ps[1]
    distancia = np.sqrt((pe[0]-ps[0])**2 + (pe[1] - ps[1])**2)
    normal_se = np.array((-dy, dx))/distancia #ou (dy, -dx) --> vetor normal à (pe-ps)
    max_vd = 500 # distancia máxima em que os pontos de controle serão sorteados. Exemplo: 1 gera traçados retos. Usar números maiores que 100.
    n_points = 6 # numero de pontos de controle entre pe e ps. Quanto maior esse número mais curvas são geradas.

    control_points = []
    hds = np.linspace(0.2, 0.8, n_points) # faz com que os pontos de controle sejam equidistantes em relação à (pe-ps)

    for j in range(n_points):            
        control_point = ((pe-ps) * hds[j]) # setar as distancias horizontais dessa maneira deixa um aspecto mais natural
        control_point += (normal_se * np.random.uniform(low=-1, high=1) * max_vd)
        control_points.append(control_point+ps)

    control_points.insert(0, ps)
    control_points.append(pe)
    curve = bezier(control_points, precision=100)       
    array_curves.append(curve)        
    return array_curves 

In [58]:
from matplotlib import pyplot as plt
import sys
from Funcoes_gerais import funcoes
from pathlib import Path

#linux
#sys.path.insert(0, "/home/adriano/projeto_mestrado/modules/")

#windows
sys.path.insert(0, r"C:\Users\adria\Documents\Mestrado\texture_codes\modules")

#root_dir = f"/home/adriano/projeto_mestrado/modules"
root_dir = Path(r"C:\Users\adria\Documents\Mestrado\texture_codes\modules")
tracados_bezier = f'{root_dir}/Artificial_Lines/tracados_bezier/'

array_curves = []
for i in range(5): 
    pontos, distancia = criando_pontos(25)
    curve = criar_array_curves(pontos)
    dict = {
        "curve": curve,
        "distancia": distancia
    }
    curve.append(distancia)   
    funcoes.gravar_dict_arquivo(dict,f'{tracados_bezier}/teste/img_savedata_{i+5}.json')
    array_curves.append(curve)

In [ ]:
from Background import background_generation as backgen
from geopandas import gpd

for i in range(len(array_curves)):    

    linha_central,linha_offset_esquerda,linha_offset_direita, maior_tamanho = backgen.retorna_linhas_offset_posicao_tamanho(array_curves[i][0],30)   
    fig, ax2 = plt.subplots(figsize=(10,5))
    gp4 = gpd.GeoSeries([linha_offset_esquerda, linha_central, linha_offset_direita])   
    gp4.plot(ax=ax2, cmap="tab10")   
    fig.savefig(f'{root_dir}/Artificial_Lines/LineStrings/varios_pontos_controle/tracado_{i}.svg', format='svg')


In [8]:
import numpy as np

pontos, distancia = criando_pontos(25)


array_curves = []
ps = pontos[0][0] # ponto inicial   
pe = pontos[0][1] # ponto final 
dx = pe[0]-ps[0]
dy = pe[1]-ps[1]
distancia = np.sqrt((pe[0]-ps[0])**2 + (pe[1] - ps[1])**2)
normal_se = np.array((-dy, dx))/distancia #ou (dy, -dx) --> vetor normal à (pe-ps)
max_vd = 500 # distancia máxima em que os pontos de controle serão sorteados. Exemplo: 1 gera traçados retos. Usar números maiores que 100.
n_points = 6 # numero de pontos de controle entre pe e ps. Quanto maior esse número mais curvas são geradas.

control_points = []
hds = np.linspace(0.2, 0.8, n_points) # faz com que os pontos de controle sejam equidistantes em relação à (pe-ps)

for j in range(n_points):            
    control_point = ((pe-ps) * hds[j]) # setar as distancias horizontais dessa maneira deixa um aspecto mais natural
    control_point += (normal_se * np.random.uniform(low=-1, high=1) * max_vd)
    control_points.append(control_point+ps)

control_points.insert(0, ps)
control_points.append(pe)
#curve = bezier(control_points, precision=100)       
#array_curves.append(curve) 

#pontos, distancia = criando_pontos(25)
#curve = criar_array_curves(pontos)

ts = np.linspace(0, 1, 100)
print(ts)

B = np.zeros((len(ts), 2), dtype=np.float64)

# gera uma matriz com duas colunas e a quantidade de linhas dependendo do tamanho de ts de zeros, com o tipo float
#print(B)

n = len(control_points) - 1

for idx, t in enumerate(ts):        
    for i in range(n+1):
        # 
        bin_coef = np.math.factorial(n) / (np.math.factorial(i) * np.math.factorial(n-i))
        print(bin_coef)
        
        Pin = bin_coef * (1-t)**(n-i) * t**i
        print(Pin)

        B[idx] += Pin * control_points[i]
        print(B)
# retorna em B todos os pontos mutliplicado por Pin 


[0.         0.01010101 0.02020202 0.03030303 0.04040404 0.05050505
 0.06060606 0.07070707 0.08080808 0.09090909 0.1010101  0.11111111
 0.12121212 0.13131313 0.14141414 0.15151515 0.16161616 0.17171717
 0.18181818 0.19191919 0.2020202  0.21212121 0.22222222 0.23232323
 0.24242424 0.25252525 0.26262626 0.27272727 0.28282828 0.29292929
 0.3030303  0.31313131 0.32323232 0.33333333 0.34343434 0.35353535
 0.36363636 0.37373737 0.38383838 0.39393939 0.4040404  0.41414141
 0.42424242 0.43434343 0.44444444 0.45454545 0.46464646 0.47474747
 0.48484848 0.49494949 0.50505051 0.51515152 0.52525253 0.53535354
 0.54545455 0.55555556 0.56565657 0.57575758 0.58585859 0.5959596
 0.60606061 0.61616162 0.62626263 0.63636364 0.64646465 0.65656566
 0.66666667 0.67676768 0.68686869 0.6969697  0.70707071 0.71717172
 0.72727273 0.73737374 0.74747475 0.75757576 0.76767677 0.77777778
 0.78787879 0.7979798  0.80808081 0.81818182 0.82828283 0.83838384
 0.84848485 0.85858586 0.86868687 0.87878788 0.88888889 0.89898